To do:
* Write a class for preprocessing in a pipeline:
    * Add the option to remove the NOX variable.
    * Apply logarithm to PTRATIO, INDUS, and TAX.
* Use only top predictors (corr >= 0.5)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pprint import pprint
from math import ceil

import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import make_pipeline 

from sklearn.preprocessing import MinMaxScaler

In [2]:
def rm_outliers_below(df, columns):
    """Returns its input dataframe without the lower outliers of the features given in the columns iterable."""
    df2 = df.copy()
    if type(df) == 'pandas.core.frame.DataFrame':
        for column in columns:
            Q1 = df2[column].quantile(0.25)
            Q3 = df2[column].quantile(0.75)
            mult = 1.5*(Q3-Q1)
            outlier_ix = df2[df2[column] <= Q1 - mult].index
            df2.drop(index=outlier_ix, inplace = True)
    elif type(df) == 'pandas.core.series.Series':
        Q1 = df2.quantile(0.25)
        Q3 = df2.quantile(0.75)
        mult = 1.5*(Q3-Q1)
        outlier_ix = df2[df2 <= Q1 - mult].index
        df2.drop(index=outlier_ix, inplace = True)
        
    return df2

In [3]:
def rm_outliers_above(df, columns):
    """Returns its input dataframe without the upper outliers of the features given in the columns iterable."""
    df2 = df.copy()
    for column in columns:
        if type(df) == 'pandas.core.frame.DataFrame':
            Q1 = df2[column].quantile(0.25)
            Q3 = df2[column].quantile(0.75)
            mult = 1.5*(Q3-Q1)
            outlier_ix = df2[df2[column] >= Q3 + mult].index
            df2.drop(index=outlier_ix, inplace = True)
        elif type(df) == 'pandas.core.series.Series':
            Q1 = df2.quantile(0.25)
            Q3 = df2.quantile(0.75)
            mult = 1.5*(Q3-Q1)
            outlier_ix = df2[df2 >= Q3 + mult].index
            df2.drop(index=outlier_ix, inplace = True)
        
    return df2

In [4]:
def reduce_skew(df):
    """Applies the logarithm to the columns with a skewness greater than 0.5 (not absolute value)."""
    df2 = df.copy()
    if type(df) == 'pandas.core.frame.DataFrame':
        for column in df.columns:
            if df2[column].skew() >= 0.5:
                df2[column] = np.log(df2[column])
    elif type(df) == 'pandas.core.series.Series':
        if df2.skew() >= 0.5:
                df2 = np.log(df2)
    
    return df2

In [5]:
class FeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, poly_features=None, max_degree=3, rm_outliers=None):
        self.poly_features = poly_features #List of features to make polynomial
        self.max_degree = max_degree #Maximum polynomial degree
        self.rm_outliers = rm_outliers #Features to remove upper outliers from
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_temp = X.copy()
        #Remove outliers
        if self.rm_outliers is not None:
            X_temp = rm_outliers_above(X_temp, self.rm_outliers)
        #Apply log to reduce skew
        X_temp = reduce_skew(X_temp)
        #Make polynomial features
        if self.poly_features is not None:
            poly_feat_transformer = PolynomialFeatures(degree=self.max_degree)
            X_temp = poly_feat_transformer.fit_transform(X_temp[self.poly_features])
            
        return X_temp

In [6]:
def pipeline_grid_search_eval( X_train, y_train, pipe, param_grid):
    """Runs the given datasets through a pipeline, performs a grid search and evaluates its performance through
    cross-validation and mean squared error.
    
    Arguments:
    -pipeline: Sklearn pipeline object. An ML model must be trained at some step in the pipe.
    -param_grid: List of dictionaries with the hyperparamaters to try in the grid search.
    -X_train: NumPy array with the features to be trained on.
    -y_train: NumPy array with the labels used to train and evaluate.
    
    Returns:
    search_res: The fit GridSearchCV
    """
    grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    search_res = grid_search.fit(X_train, y_train)
    
    best_cv_score = np.sqrt(-search_res.best_score_)
    best_model = search_res.best_estimator_
    best_train_score = np.sqrt(mean_squared_error(y_train, best_model.predict(X_train)))
    
    pprint('Best paramaters:\n' + str(search_res.best_params_))
    print('\n')
    print('''CV error: %.2f
train error: %.2f
Label mean for comparison: %.2f'''%(best_cv_score, best_train_score, y_train.mean()))
    
    return search_res

## Obtaining train and test sets

In [7]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [8]:
X_train.shape

(404, 13)

# First model

I'll apply a simple first model to get an idea of how it well it performs. I'll use all features with a correlation coefficient with the target greater than 0.5, except for the RAD variable due to its extremely high correlation with TAX.

In [9]:
X2 = pd.merge(X_train, y_train, left_index = True, right_index=True)

In [10]:
#Removing outliers from y2_train
#X2 = rm_outliers_above(X2, ['target'])
outlier_ix = X2[X2['target'] >= 50].index
X2.drop(index=outlier_ix, inplace=True)

In [11]:
predictors = ['LSTAT', 'INDUS', 'NOX', 'PTRATIO', 'RM', 'TAX', 'DIS', 'AGE', 'CRIM']
X2_train = X2[predictors].copy()
y2_train = X2['target'].copy()

In [12]:
X_feat_transformer = FeatureTransformer()
standard_scaler = StandardScaler()
l_reg = LinearRegression()
pipe = make_pipeline(X_feat_transformer,
    standard_scaler,
    l_reg
)
#X2_train_stand = scaler.fit_transform(X2_train)
#X2_train_stand = pd.DataFrame(X2_train_stand, columns=X2_train.columns)

In [13]:
#pipe.fit(X2_train, y2_train)

In [15]:
for col in X2_train.columns:
    if np.abs(X_train[col].skew()) > 0.5:
        X2_train[col] = np.log1p(X2_train[col])
X2_train = standard_scaler.fit_transform(X2_train)
y2_train = np.log1p(y2_train)

In [16]:
cv_scores = cross_val_score(l_reg, X2_train, y2_train, cv=5, scoring='neg_mean_squared_error')
cv_score = np.sqrt(-cv_scores.mean())
cv_std = cv_scores.std()

In [17]:
l_reg.fit(X2_train, y2_train)
train_score = np.sqrt(mean_squared_error(y2_train, l_reg.predict(X2_train)))

In [18]:
print('CV score: %.2f +/- %.2f\ntrain score: %.2f\nLabel mean for comparison: %.2f'%(cv_score, cv_std, train_score, y_train.mean()))

CV score: 0.19 +/- 0.00
train score: 0.18
Label mean for comparison: 22.80


Since the training set error is smaller than the CV error, the model is overfitting. I'll try to engineer a couple of features next.

# Feature engineering

I'll perform a quick grid search to figure out if it's worth it to apply a polynomial transformation to the LSTAT variable. Also, I'll try several combinations of features to apply a log to.

In [31]:
least_corr_features = list(np.abs(corr_matrix['target']).sort_values().index[:4])

In [32]:
poly_features = [predictors]
#drop_features= [None, 
 #               ['NOX'],
  #              ['NOX', 'TAX', 'LSTAT']]
param_grid = [
    {'feature_transformer__poly_features':poly_features,
     'feature_transformer__max_degree':list(range(2,5))
    }
]

In [33]:
pipe = Pipeline([
    ('feature_transformer', FeatureTransformer()),
    ('standardization', StandardScaler()),
    ('linear_reg', LinearRegression())
])

In [39]:
pipeline_grid_search_eval(X2_train, y_train, pipe, param_grid)

('Best paramaters:\n'
 "{'feature_transformer__max_degree': 2, 'feature_transformer__poly_features': "
 "['LSTAT', 'RM', 'PTRATIO', 'INDUS', 'TAX']}")


CV error: 3.24
train error: 2.88
Label mean for comparison: 20.70


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_transformer',
                                        FeatureTransformer()),
                                       ('standardization', StandardScaler()),
                                       ('linear_reg', LinearRegression())]),
             n_jobs=-1,
             param_grid=[{'feature_transformer__max_degree': [2, 3, 4],
                          'feature_transformer__poly_features': [['LSTAT', 'RM',
                                                                  'PTRATIO',
                                                                  'INDUS',
                                                                  'TAX']]}],
             scoring='neg_mean_squared_error')

With the new polynomial and logarithmic features, the train score diminished considerably, and the CV score also went down a little. However, the model is now overfitting, so I'll try regularization.

In [40]:
from sklearn.linear_model import Ridge

In [41]:
reg_param_grid = [
    {'feature_transformer__poly_features':poly_features,
     'feature_transformer__max_degree':list(range(4,6)),
     'ridge_reg__alpha':np.arange(9,15)}
]

In [42]:
reg_pipe = Pipeline([
    ('feature_transformer', FeatureTransformer()),
    ('standardization', StandardScaler()),
    ('ridge_reg', Ridge())
])

In [43]:
pipeline_grid_search_eval(X2_train, y_train, reg_pipe, reg_param_grid)

('Best paramaters:\n'
 "{'feature_transformer__max_degree': 4, 'feature_transformer__poly_features': "
 "['LSTAT', 'RM', 'PTRATIO', 'INDUS', 'TAX'], 'ridge_reg__alpha': 14}")


CV error: 3.19
train error: 2.91
Label mean for comparison: 20.70


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_transformer',
                                        FeatureTransformer()),
                                       ('standardization', StandardScaler()),
                                       ('ridge_reg', Ridge())]),
             n_jobs=-1,
             param_grid=[{'feature_transformer__max_degree': [4, 5],
                          'feature_transformer__poly_features': [['LSTAT', 'RM',
                                                                  'PTRATIO',
                                                                  'INDUS',
                                                                  'TAX']],
                          'ridge_reg__alpha': array([ 9, 10, 11, 12, 13, 14])}],
             scoring='neg_mean_squared_error')

It's still overfitting. Perhaps trying a less complicated model will fix it

# Support vector regressor

In [48]:
svr_pipe = Pipeline([
    ('feature_transformer', FeatureTransformer()),
    ('standardization', StandardScaler()),
    ('SVR', SVR())
])

#Defining a new parameter grid for the SVR
svr_param_grid = [
    {'feature_transformer__poly_features':poly_features,
     'feature_transformer__max_degree':list(range(4,6)),
     'SVR__gamma':np.arange(1,11),
     'SVR__C':np.arange(10,21),
     'SVR__kernel':['linear','rbf']}
]

In [50]:
search_res = pipeline_grid_search_eval( X2_train, y_train, svr_pipe, svr_param_grid)

"Best paramaters:\n{'SVR__C': 11, 'SVR__gamma': 1, 'SVR__kernel': 'rbf'}"


CV error: 3.49
train error: 2.29
Label mean for comparison: 20.70


This one performs actively worse than the linear model, though at least it doesn't overfit as badly. Of note is also that with this model the set of features to apply the log to is different.

# Random forest regressor

In [51]:
forest_pipe = Pipeline([
    ('feature_transformer', FeatureTransformer()),
    ('standardization', StandardScaler()),
    ('random_forest', RandomForestRegressor(n_estimators=200,max_features='sqrt',random_state=42, n_jobs=-1))
])

#Defining a new parameter grid for the SVR
forest_param_grid = [
    {'feature_transformer__poly_features':poly_features,
     'feature_transformer__max_degree':list(range(2,5)),
     #'random_forest__n_estimators': np.arange(100,301,100),
     'random_forest__max_depth':np.arange(10,15),
     #'random_forest__max_leaf_nodes':np.arange(95, 100),
     'random_forest__max_features':['auto', 'sqrt', 'log2']
     }
]

In [53]:
search_res = pipeline_grid_search_eval( X2_train, y_train, forest_pipe, forest_param_grid)

('Best paramaters:\n'
 "{'feature_transformer__max_degree': 2, 'feature_transformer__poly_features': "
 "['LSTAT', 'RM', 'PTRATIO', 'INDUS', 'TAX'], 'random_forest__max_depth': 10, "
 "'random_forest__max_features': 'sqrt'}")


CV error: 3.16
train error: 1.27
Label mean for comparison: 20.70
